In [3]:
import json

with open('/kaggle/input/emosim508/EmoSim508.json', encoding='utf8') as f:
    dataset = json.load(f)
#     .replace(r'\\', '\\'))
dataset[0]

{'emojiPairId': '199',
 'emojiPair': {'emojiOne': {'unicodelong': '\\U0001F3B5',
   'unicodeshort': 'U+1F3B5',
   'title': 'musical note'},
  'emojiTwo': {'unicodelong': '\\U0001F3B6',
   'unicodeshort': 'U+1F3B6',
   'title': 'musical notes'}},
 'emojiPairSimilarity': {'Google_Sense_Label': 0.94,
  'Twitter_Sense_Def': 1.0,
  'Google_Sense_All': 1.0,
  'Google_Sense_Def': 1.0,
  'Google_Sense_Desc': 0.84,
  'Twitter_Sense_All': 1.0,
  'Twitter_Sense_Desc': 0.89,
  'Twitter_Sense_Label': 0.92,
  'Human_Annotator_Agreement': 4.0}}

In [6]:
from collections import defaultdict

googleSense = []
emojis = set()
description = defaultdict()
for x in dataset:
    emo1 = x['emojiPair']['emojiOne']['unicodelong'].encode('utf_8').decode('unicode_escape')
    emo2 = x['emojiPair']['emojiTwo']['unicodelong'].encode('utf_8').decode('unicode_escape')
    score = x['emojiPairSimilarity']['Google_Sense_Label']
    googleSense.append((emo1, emo2, score))
    emojis.update((emo1, emo2))
    description[emo1] = x['emojiPair']['emojiOne']['title']
    description[emo2] = x['emojiPair']['emojiTwo']['title']

In [7]:
len(emojis), len(googleSense)

(158, 508)

In [8]:
emojis = list(emojis)
texts = [description[emo] for emo in emojis]

In [9]:
import numpy as np

G = np.eye((len(emojis)))
for emo1, emo2, score in googleSense:
    G[emojis.index(emo1), emojis.index(emo2)] = score
    G[emojis.index(emo2), emojis.index(emo1)] = score

In [27]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.rcParams['text.latex.unicode'] = True
# plt.figure(figsize=(15, 15))


# sns.clustermap(G,
#            cmap=sns.diverging_palette(230, 20, as_cmap=True))
# #            xticklabels=emojis)

In [26]:
from sklearn.cluster import AgglomerativeClustering


clust = AgglomerativeClustering(distance_threshold=0.2, n_clusters=None,
                               affinity='precomputed',
#                                linkage='complete')
                               linkage='average')
#                                linkage='single')
labels = clust.fit_predict(1-G)

clusters = {label: [emo for emo, l in zip(emojis, labels) if label == l] for label in range(max(labels))}

clusters = {label: cluster for label, cluster in clusters.items() if len(cluster) > 1}
clusters
# for i in range(10):    # max(labels)
#     print('Cluster', i)
#     for emo, label in zip(emojis, labels):
#         if label == i:
#             print(emo, end=' ')
#     print()

{0: ['🙌', '👏'],
 1: ['😭', '😢'],
 2: ['🎉', '🎈', '🎊'],
 3: ['💖', '💗'],
 4: ['☺', '😊'],
 5: ['💛', '💙', '💚', '💜'],
 6: ['⬅', '➡'],
 7: ['💘', '❤'],
 12: ['🙉', '🙊', '🙈'],
 13: ['😂', '😆'],
 15: ['💞', '💕'],
 25: ['👇', '👉'],
 26: ['💋', '😘', '😍'],
 52: ['🎶', '🎵']}

In [14]:
from scipy.cluster.hierarchy import dendrogram



In [15]:
from sklearn.manifold import MDS

mds = MDS(dissimilarity='precomputed', n_init=20, max_iter=3000, eps=1e-5, random_state=876)
G2d = mds.fit_transform(1-G)
G2d.shape, mds.stress_

((158, 2), 1952.7509735412084)

In [20]:
import pandas as pd

df = pd.DataFrame(G2d, columns=['x', 'y'])
df['emo'] = emojis
df['cluster'] = labels
df['text'] = texts
df

,x,y,emo,cluster,text
0,0.535419,-0.033934,🎼,112,musical score
1,-0.283888,0.278182,🙏,131,folded hands
2,-0.222162,0.576018,✨,138,sparkles
3,0.433321,-0.633461,🖖,98,vulcan salute
4,-0.778502,-0.104911,🌚,127,new moon face
...,...,...,...,...,...
153,-0.587379,-0.503287,🕹,28,joystick
154,0.152486,0.744986,💫,11,dizzy
155,0.119718,-0.751177,🦄,8,unicorn face
156,0.062912,0.347171,👉,25,backhand index pointing right


In [21]:
import plotly.express as px

px.scatter(data_frame=df, x='x', y='y', text='emo', hover_name='text')


In [37]:
# sns.clustermap(1-G)